In [1]:
import xlwings as xw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
xw.App.DisplayAlerts = False

# Current progress 9/2/2020
- All cell locations have been switched over from Count to Balance, and I've reviewed the code to make sure it works for balances now instead of loan counts
- All data extraction and cleaning is DONE, and the output dataframe has been saved as delinquency_stats_balances.pkl in my Data Repository folder

# Stage 1: Data extraction functions

In [2]:
# Handles January and February 2007
def extract_jan07_feb07(month):
    report_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        '2007 Through May 2013' + '/'
    filename = ''
    if month == 'jan':
        filename = report_loc + 'bear-stearns-2006-he10-investor-report-01-25-2007.xls'
    elif month == 'feb':
        filename = report_loc + 'bear-stearns-2006-he10-investor-report-02-26-2007.xls'
    investor_rpt = xw.Book(filename)
    LSS_part_one = investor_rpt.sheets['Loan Status Summary Part I']
    jan_or_feb = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
    'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
    'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
    'Foreclosure (Balance)','REO (Count)', 'REO (Balance)','Total Mortgage Principal Remaining'])
    locations = np.arange(30, 609, 34)
    for row_num in locations:
        row_loc = "A" + str(row_num) + ":O" + str(row_num)
        name_loc = "A" + str(row_num-1)
        current_data = np.concatenate((LSS_part_one.range(row_loc).options(np.array).value,\
            np.array(['TMPR PLACEHOLDER'])))
        group_name = LSS_part_one.range(name_loc).value
        jan_or_feb.loc[group_name] = current_data
        # Compute the total mortgage principal remaining in the pool
        curr_num = LSS_part_one.range('C' + str(row_num-17)).value
        one_delinq_num = LSS_part_one.range('E' + str(row_num-17)).value
        two_delinq_num = LSS_part_one.range('G' + str(row_num-17)).value
        three_plus_delinq_num = LSS_part_one.range('I' + str(row_num-17)).value
        bankruptcy_num = LSS_part_one.range('K' + str(row_num-17)).value
        foreclosure_num = LSS_part_one.range('M' + str(row_num-17)).value
        reo_num = LSS_part_one.range('O' + str(row_num-17)).value
        jan_or_feb.at[group_name,'Total Mortgage Principal Remaining'] = curr_num + one_delinq_num + two_delinq_num + \
            three_plus_delinq_num + bankruptcy_num + foreclosure_num + reo_num
    investor_rpt.close()
    jan_or_feb.index.rename('Group',inplace=True)
    return jan_or_feb

In [3]:
# Handles April 2007
def extract_apr07():
    report_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        '2007 Through May 2013' + '/'
    filename = report_loc + 'bear-stearns-2006-he10-investor-report-04-25-2007.xls'
    investor_rpt = xw.Book(filename)
    LSS_part_one = investor_rpt.sheets['Loan Status Summary Part I']
    apr07 = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
    'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
    'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
    'Foreclosure (Balance)','REO (Count)', 'REO (Balance)','Total Mortgage Principal Remaining'])
    locations = np.arange(11, 281, 15)
    for row_num in locations:
        row_loc = "A" + str(row_num) + ":O" + str(row_num)
        name_loc = "A" + str(row_num-1)
        current_data = np.concatenate((LSS_part_one.range(row_loc).options(np.array).value,\
            np.array(['TMPR PLACEHOLDER'])))
        group_name = LSS_part_one.range(name_loc).value
        apr07.loc[group_name] = current_data
        # Compute the total mortgage principal remaining in the pool
        curr_num = LSS_part_one.range('C' + str(row_num-6)).options(np.array).value   
        one_delinq_num = LSS_part_one.range('E' + str(row_num-6)).options(np.array).value
        two_delinq_num = LSS_part_one.range('G' + str(row_num-6)).options(np.array).value
        three_plus_delinq_num = LSS_part_one.range('I' + str(row_num-6)).options(np.array).value
        bankruptcy_num = LSS_part_one.range('K' + str(row_num-6)).options(np.array).value
        foreclosure_num = LSS_part_one.range('M' + str(row_num-6)).options(np.array).value
        reo_num = LSS_part_one.range('O' + str(row_num-6)).options(np.array).value
        apr07.at[group_name,'Total Mortgage Principal Remaining'] = curr_num + one_delinq_num + two_delinq_num + \
            three_plus_delinq_num + bankruptcy_num + foreclosure_num + reo_num
    investor_rpt.close()
    apr07.index.rename('Group',inplace=True)
    return apr07

In [4]:
# The core data extraction function for January 2007 to May 2013
# This reads the LSS Part I sheet from every investor report in the given subfolder of my Office directory
def extract_jan07_may13():
    reports_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        '2007 Through May 2013' + '/'
    reports_dir = os.listdir(reports_loc)
    reports_dir.sort(key=lambda z: z[-8:]+z[-14:-12])
    # Create a list to hold the monthly dataframes, which we'll concatenate at the end
    data_list = [x for x in range(len(reports_dir) - 1)]
    for i in range(len(reports_dir) - 1):
        filename = reports_loc + reports_dir[i]
        # Deal with the unique formats of the January, February and April 2007 reports
        if reports_dir[i] == 'bear-stearns-2006-he10-investor-report-01-25-2007.XLS':
            data_list[i] = extract_jan07_feb07('jan')
            continue
        elif reports_dir[i] == 'bear-stearns-2006-he10-investor-report-02-26-2007.XLS':
            data_list[i] = extract_jan07_feb07('feb')
            continue
        elif reports_dir[i] == 'bear-stearns-2006-he10-investor-report-04-25-2007.XLS':
            data_list[i] = extract_apr07()
            continue
        investor_rpt = xw.Book(filename)
        LSS_part_one = investor_rpt.sheets['Loan Status Summary Part I']
        # Set up a dataframe to hold an entire month's numbers
        one_month = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
        'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
        'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
        'Foreclosure (Balance)','REO (Count)', 'REO (Balance)','Total Mortgage Principal Remaining'])
        # The row numbers of the current-month percentage figures we're looking for
        locations = [30, 67, 104, 141, 178, 215, 252, 289, 326, 363, 400, 437, 474, 511, 548, 585, 622, 659]
        # For each row, grab all the data, and put it into the dataframe using the loan group name as the index value
        for row_num in locations:
            row_loc = "A" + str(row_num) + ":O" + str(row_num)
            name_loc = "A" + str(row_num-1) + ":A" + str(row_num-1)
            current_data = np.concatenate((LSS_part_one.range(row_loc).options(np.array).value,\
            np.array(['TMPR PLACEHOLDER'])))
            group_name = LSS_part_one.range(name_loc).value
            one_month.loc[group_name] = current_data
            # Find the total mortgage principal still remaining in the deal that month
            curr_num = LSS_part_one.range('C' + str(row_num-17)).options(np.array).value   
            one_delinq_num = LSS_part_one.range('E' + str(row_num-17)).options(np.array).value
            two_delinq_num = LSS_part_one.range('G' + str(row_num-17)).options(np.array).value
            three_plus_delinq_num = LSS_part_one.range('I' + str(row_num-17)).options(np.array).value
            bankruptcy_num = LSS_part_one.range('K' + str(row_num-17)).options(np.array).value
            foreclosure_num = LSS_part_one.range('M' + str(row_num-17)).options(np.array).value
            reo_num = LSS_part_one.range('O' + str(row_num-17)).options(np.array).value
            one_month.at[group_name,'Total Mortgage Principal Remaining'] = curr_num + one_delinq_num + two_delinq_num + \
                three_plus_delinq_num + bankruptcy_num + foreclosure_num + reo_num
        # Put the current month's dataframe into the proper spot in the 'master list'
        data_list[i] = one_month
        investor_rpt.close()
        
    data_df = pd.concat(data_list)
    # Set up the MultiIndex of the final output dataframe
    data_df.index.rename('Group',inplace=True)
    data_df.set_index('Date',append=True,inplace=True)
    data_df = data_df.reorder_levels(['Date','Group'])
    return data_df

In [5]:
def extract_new_format_month(reports_loc, filename):
    full_path = reports_loc + filename
    investor_rpt = xw.Book(full_path)
    delinq_summary = investor_rpt.sheets['Delinquency Summary _ Lien and ']
    # Set up a dataframe to hold the entire month's numbers
    one_month = pd.DataFrame(columns=['Date','Current (Count)', 'Current (Balance)', 'Delinq 1 Month (Count)', \
        'Delinq 1 Month (Balance)', 'Delinq 2 Months (Count)', 'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Count)', \
        'Delinq 3+ Months (Balance)', 'Bankruptcy (Count)', 'Bankruptcy (Balance)', 'Foreclosure (Count)', \
        'Foreclosure (Balance)','REO (Count)', 'REO (Balance)'],index=['Total - All Loans','Total - Fixed', \
        'Total - ARM'])
    
    # Find the correct sheet locations depending on the date of the investor report being accessed
    if int(filename[-8:-4]) < 2014 or (int(filename[-8:-4]) == 2014 and int(filename[-14:-12]) < 3):
        # June 2013 - February 2014
        # Create dictionaries of values to be extracted for each loan type and their locations in the investor report
        all_loans_dict = {'Current (Balance)':'AD113','Delinq 1 Month (Balance)':'M115','Delinq 2 Months (Balance)':'Q115', \
            'Delinq 3+ Months (Balance)':['U115','Y115'],'Bankruptcy (Balance)':'AD116','Foreclosure (Balance)': \
            'AD117','REO (Balance)':'AD118'}
        # Find the total principal balance of mortgages
        all_loans_denom = delinq_summary.range('AD119').value
        
        fixed_rate_dict = {'Current (Balance)':'X146','Delinq 1 Month (Balance)':'I147','Delinq 2 Months (Balance)':'L147', \
            'Delinq 3+ Months (Balance)':['P147','S147'],'Bankruptcy (Balance)':'X148','Foreclosure (Balance)': \
            'X149','REO (Balance)':'X150'}
        # Find the total principal balance of fixed-rate mortgages
        fixed_rate_denom = sum(delinq_summary.range('X146:X150').value)
        
        adj_rate_dict = {'Current (Balance)':'X140','Delinq 1 Month (Balance)':'I142','Delinq 2 Months (Balance)':'L142', \
            'Delinq 3+ Months (Balance)':['P142','S142'],'Bankruptcy (Balance)':'X143','Foreclosure (Balance)': \
            'X144','REO (Balance)':'X145'}
        # Find the total principal balance of adjustable-rate mortgages
        adj_rate_denom = sum(delinq_summary.range('X142:X145').value) + delinq_summary.range('X140').value
    elif int(filename[-8:-4]) < 2015 or (int(filename[-8:-4]) == 2015 and int(filename[-14:-12]) < 10):
        # March 2014 - September 2015
        all_loans_dict = {'Current (Balance)':'AG109','Delinq 1 Month (Balance)':'P111','Delinq 2 Months (Balance)':'T111', \
            'Delinq 3+ Months (Balance)':['X111','AB111'],'Bankruptcy (Balance)':'AG112','Foreclosure (Balance)': \
            'AG113','REO (Balance)':'AG114'}
        all_loans_denom = delinq_summary.range('AG115').value
        fixed_rate_dict = {'Current (Balance)':'AC142','Delinq 1 Month (Balance)':'L143','Delinq 2 Months (Balance)':'P143', \
            'Delinq 3+ Months (Balance)':['T143','X143'],'Bankruptcy (Balance)':'AC144','Foreclosure (Balance)': \
            'AC145','REO (Balance)':'AC146'}
        fixed_rate_denom = sum(delinq_summary.range('AC142:AC146').value)
        adj_rate_dict = {'Current (Balance)':'AC136','Delinq 1 Month (Balance)':'L138','Delinq 2 Months (Balance)':'P138', \
            'Delinq 3+ Months (Balance)':['T138','X138'],'Bankruptcy (Balance)':'AC139','Foreclosure (Balance)': \
            'AC140','REO (Balance)':'AC141'}
        adj_rate_denom = sum(delinq_summary.range('AC138:AC141').value) + delinq_summary.range('AC136').value
    elif int(filename[-8:-4]) < 2016 or (int(filename[-8:-4]) == 2016 and int(filename[-14:-12]) < 5):
        # October 2015 - April 2016
        all_loans_dict = {'Current (Balance)':'AH117','Delinq 1 Month (Balance)':'R119','Delinq 2 Months (Balance)':'V119', \
            'Delinq 3+ Months (Balance)':['Z119','AD119'],'Bankruptcy (Balance)':'AH120','Foreclosure (Balance)': \
            'AH121','REO (Balance)':'AH122'}
        all_loans_denom = delinq_summary.range('AH123').value
        fixed_rate_dict = {'Current (Balance)':'AB154','Delinq 1 Month (Balance)':'L155','Delinq 2 Months (Balance)':'P155', \
            'Delinq 3+ Months (Balance)':['T155','X155'],'Bankruptcy (Balance)':'AB156','Foreclosure (Balance)': \
            'AB157','REO (Balance)':'AB158'}
        fixed_rate_denom = sum(delinq_summary.range('AB154:AB158').value)
        adj_rate_dict = {'Current (Balance)':'AB147','Delinq 1 Month (Balance)':'L150','Delinq 2 Months (Balance)':'P150', \
            'Delinq 3+ Months (Balance)':['T150','X150'],'Bankruptcy (Balance)':'AB151','Foreclosure (Balance)': \
            'AB152','REO (Balance)':'AB153'}
        adj_rate_denom = sum(delinq_summary.range('AB150:AB153').value) + delinq_summary.range('AB147').value
    else:
        # May 2016 - March 2020
        all_loans_dict = {'Current (Balance)':'AF117','Delinq 1 Month (Balance)':'P119','Delinq 2 Months (Balance)':'T119', \
            'Delinq 3+ Months (Balance)':['X119','AB119'],'Bankruptcy (Balance)':'AF120','Foreclosure (Balance)': \
            'AF121','REO (Balance)':'AF122'}
        all_loans_denom = delinq_summary.range('AF123').value
        fixed_rate_dict = {'Current (Balance)':'AB154','Delinq 1 Month (Balance)':'L155','Delinq 2 Months (Balance)':'P155', \
            'Delinq 3+ Months (Balance)':['T155','X155'],'Bankruptcy (Balance)':'AB156','Foreclosure (Balance)': \
            'AB157','REO (Balance)':'AB158'}
        fixed_rate_denom = sum(delinq_summary.range('AB154:AB158').value)
        adj_rate_dict = {'Current (Balance)':'AB147','Delinq 1 Month (Balance)':'L150','Delinq 2 Months (Balance)':'P150', \
            'Delinq 3+ Months (Balance)':['T150','X150'],'Bankruptcy (Balance)':'AB151','Foreclosure (Balance)': \
            'AB152','REO (Balance)':'AB153'}
        adj_rate_denom = sum(delinq_summary.range('AB150:AB153').value) + delinq_summary.range('AB147').value
    
    # Add on a column to track the total mortgage principal remaining in the pool
    one_month['Total Mortgage Principal Remaining'] = 'ERROR' # If this doesn't get filled with a number, I'll be able to see that
    # Loop through all of the columns we want to retrieve data for and access their respective sheet locations
    for field in all_loans_dict.keys():
        if field == 'Delinq 3+ Months (Balance)':
            months_3 = delinq_summary.range(all_loans_dict[field][0]).value
            months_4_or_more = delinq_summary.range(all_loans_dict[field][1]).value
            one_month.loc['Total - All Loans'][field] = (months_3 + months_4_or_more) / all_loans_denom
        else:
            one_month.loc['Total - All Loans'][field] = delinq_summary.range(all_loans_dict[field]).value \
                      / all_loans_denom
        one_month.loc['Total - All Loans']['Total Mortgage Principal Remaining'] = all_loans_denom
    # Perform the same task, but for the fixed-rate mortgages only
    for field in fixed_rate_dict.keys():
        if field == 'Delinq 3+ Months (Balance)':
            months_3 = delinq_summary.range(fixed_rate_dict[field][0]).value
            months_4_or_more = delinq_summary.range(fixed_rate_dict[field][1]).value
            one_month.loc['Total - Fixed'][field] = (months_3 + months_4_or_more) / fixed_rate_denom
        else:
            one_month.loc['Total - Fixed'][field] = delinq_summary.range(fixed_rate_dict[field]).value \
                      / fixed_rate_denom
        one_month.loc['Total - Fixed']['Total Mortgage Principal Remaining'] = fixed_rate_denom
    # Get data one more time, but for the adjustable-rate mortgages only
    for field in adj_rate_dict.keys():
        if field == 'Delinq 3+ Months (Balance)':
            months_3 = delinq_summary.range(adj_rate_dict[field][0]).value
            months_4_or_more = delinq_summary.range(adj_rate_dict[field][1]).value
            one_month.loc['Total - ARM'][field] = (months_3 + months_4_or_more) / adj_rate_denom
        else:
            one_month.loc['Total - ARM'][field] = delinq_summary.range(adj_rate_dict[field]).value \
                      / adj_rate_denom
        one_month.loc['Total - ARM']['Total Mortgage Principal Remaining'] = adj_rate_denom
    
    investor_rpt.close()
    one_month['Date'] = pd.to_datetime(filename[-14:-4])
    one_month.index.rename('Group',inplace=True)
    return one_month

In [6]:
def extract_new_format_all():
    reports_loc = '/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS_2006-HE10_INVESTOR_REPORTS/' + \
        'June 2013 Through March 2020' + '/'
    reports_dir = os.listdir(reports_loc)
    reports_dir.sort(key=lambda z: z[-8:]+z[-14:-12])
    # Create a list to hold the monthly dataframes, which we'll concatenate at the end
    data_list = [x for x in range(len(reports_dir) - 1)]
    for i in range(len(reports_dir) - 1):
        data_list[i] = extract_new_format_month(reports_loc, reports_dir[i])
    data_df = pd.concat(data_list)
    # Set up the MultiIndex of the final output dataframe
    data_df.set_index('Date',append=True,inplace=True)
    data_df = data_df.reorder_levels(['Date', 'Group'])
    return data_df

# Stage 2: Get the data

In [7]:
# Store all data from the various old-format extraction functions
old_format_all_data = extract_jan07_may13()

In [8]:
# Store all data from the new-format extraction function
new_format_all_data = extract_new_format_all()

In [9]:
old_format_balances_only = old_format_all_data[['Current (Balance)','Delinq 1 Month (Balance)','Delinq 2 Months (Balance)',\
    'Delinq 3+ Months (Balance)','Bankruptcy (Balance)','Foreclosure (Balance)','REO (Balance)', 'Total Mortgage Principal Remaining']]
old_format_balances_only

Current (Balance)  \
Date       Group                                                           
2007-01-25 Total [All Loans]                                      0.9890   
           Group I Loans - Total                                  0.9859   
           Group I Loans Fixed 1st Lien                           0.9890   
           Group I Loans Fixed 2nd Lien                           0.9443   
           Group I Loans 228 ARM                                  0.9858   
...                                                                  ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                     0.4806   
           Group II Loans Subgroup II 327 ARM                     0.3854   
           Group II Loans Subgroup III Fixed 1st Lien             0.5714   
           Group II Loans Subgroup III 228 ARM                    0.4624   
           Group II Loans Subgroup III 327 ARM                    0.5023   

                                                       Delinq 1 Month (Balance)  \
Date       Group                                                                  
2007-01-25 Total [All Loans]                                             0.0110   
           Group I Loans - Total                                         0.0141   
           Group I Loans Fixed 1st Lien                                  0.0110   
           Group I Loans Fixed 2nd Lien                                  0.0557   
           Group I Loans 228 ARM                                         0.0142   
...                                                                         ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                            0.0463   
           Group II Loans Subgroup II 327 ARM                            0.0437   
           Group II Loans Subgroup III Fixed 1st Lien                    0.0513   
           Group II Loans Subgroup III 228 ARM                           0.0244   
           Group II Loans Subgroup III 327 ARM                           0.0856   

                                                       Delinq 2 Months (Balance)  \
Date       Group                                                                   
2007-01-25 Total [All Loans]                                              0.0000   
           Group I Loans - Total                                          0.0000   
           Group I Loans Fixed 1st Lien                                   0.0000   
           Group I Loans Fixed 2nd Lien                                   0.0000   
           Group I Loans 228 ARM                                          0.0000   
...                                                                          ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                             0.0087   
           Group II Loans Subgroup II 327 ARM                             0.0280   
           Group II Loans Subgroup III Fixed 1st Lien                     0.0178   
           Group II Loans Subgroup III 228 ARM                            0.0306   
           Group II Loans Subgroup III 327 ARM                            0.0651   

                                                       Delinq 3+ Months (Balance)  \
Date       Group                                                                    
2007-01-25 Total [All Loans]                                               0.0000   
           Group I Loans - Total                                           0.0000   
           Group I Loans Fixed 1st Lien                                    0.0000   
           Group I Loans Fixed 2nd Lien                                    0.0000   
           Group I Loans 228 ARM                                           0.0000   
...                                                                           ...   
2013-05-28 Group II Loans Subgroup II 228 ARM                              0.0804   
           Group II Loans Subgroup II 327 ARM                              0.1134   
           Group II Loans Subgroup III Fixe

# Stage 3: Clean the old-report-format dataframe
Requirements:
- Rename "Total \[All Loans\]"/"Total (All Loans)" to "All Loans"
- Take the weighted average of all fixed-rate loan data and record it as "Fixed"
- Take the weighted average of all ARM loan data and record it as "ARM"

NEW IDEA: Each month, create a new dataframe with that month's fixed data

Then, concatenate them all together at the end (like I do with data_list in the extraction functions)

NEWER IDEA: Create an entire new dataframe to hold the TOTAL, FIXED, and ARM old-report data

The idea behind this: Iterating by month, use old_format_balances_only to calculate the relevant metrics, but assign them to their respective rows in the NEW dataframe

In [10]:
# Skeleton of creating and filling in the Fixed and ARM rows
def set_up_totals(df, month):
    # For this particular month, set up rows to contain the total fixed and ARM data
    df.loc[(month,'Fixed'),:] = np.zeros((8))
    df.loc[(month,'ARM'),:] = np.zeros((8))
    fixed_only = df[df.index.get_level_values(1).str.contains('Fixed')]
    arm_only = df[df.index.get_level_values(1).str.contains('ARM')]
    # Save the fixed- and adjustable-rate loan pool sizes for later
    fixed_count = fixed_only[fixed_only.index.get_level_values(0)==month]['Total Mortgage Principal Remaining'].sum()
    arm_count = arm_only[arm_only.index.get_level_values(0)==month]['Total Mortgage Principal Remaining'].sum()
    # Pull out the rows which are for fixed-rate loans, and those which are for adjustable-rate loans
    for col in df.columns.tolist()[:-1]:
        # For each column of data, find the weighted averages for fixed and adjustable rate loans
        fixed_weighted = fixed_only[fixed_only.index.get_level_values(0)==month][col] * \
            fixed_only[fixed_only.index.get_level_values(0)==month]['Total Mortgage Principal Remaining']
        arm_weighted = arm_only[arm_only.index.get_level_values(0)==month][col] * \
            arm_only[arm_only.index.get_level_values(0)==month]['Total Mortgage Principal Remaining']
        fixed_avg = fixed_weighted.sum() / \
            fixed_only[fixed_only.index.get_level_values(0)==month]['Total Mortgage Principal Remaining'].sum()
        arm_avg = arm_weighted.sum() / \
            arm_only[arm_only.index.get_level_values(0)==month]['Total Mortgage Principal Remaining'].sum()
        # Assign the weighted average to the general 'Fixed' and 'ARM' rows
        df.loc[(month,'Fixed'),col] = fixed_avg
        df.loc[(month,'ARM'),col] = arm_avg
    # Include the correct loan pool sizes in the 'Fixed' and 'ARM' rows
    df.at[(month,'Fixed'),'Total Mortgage Principal Remaining'] = fixed_count
    df.at[(month,'ARM'),'Total Mortgage Principal Remaining'] = arm_count

In [11]:
# Process the old-format data to get totals rows for fixed and ARM loans
old_format_deep_copy = old_format_balances_only.copy(deep=True)
months = old_format_deep_copy.index.get_level_values(0)
months = months.drop_duplicates(keep='first')
for month in months:
    set_up_totals(old_format_deep_copy, month)
# Rename the rows in the new-format dataframe to match those from the old-format data
new_format_deep_copy = new_format_all_data.copy(deep=True)
new_format_groups = new_format_deep_copy.index.get_level_values(1).tolist()
for i in range(len(new_format_groups)):
    if new_format_groups[i] == 'Total - All Loans':
        new_format_groups[i] = 'Total [All Loans]'
    elif new_format_groups[i] == 'Total - Fixed':
        new_format_groups[i] = 'Fixed'
    elif new_format_groups[i] == 'Total - ARM':
        new_format_groups[i] = 'ARM'
new_format_dates = new_format_deep_copy.index.get_level_values(0)
new_format_deep_copy.index = pd.MultiIndex.from_arrays((new_format_dates,new_format_groups))
# Remove the Count columns from the new-format data
new_format_final = new_format_deep_copy[['Current (Balance)', 'Delinq 1 Month (Balance)', \
                                'Delinq 2 Months (Balance)', 'Delinq 3+ Months (Balance)', 'Bankruptcy (Balance)', \
                                'Foreclosure (Balance)', 'REO (Balance)', 'Total Mortgage Principal Remaining']]
new_format_final = new_format_final.sort_index(level=0)
# Remove the subgroup rows from the old-format data (but first clean the old-format data's Totals index names too)
old_format_groups = old_format_deep_copy.index.get_level_values(1).tolist()
for i in range(len(old_format_groups)):
    if old_format_groups[i] == 'Total (All Loans)':
        old_format_groups[i] = 'Total [All Loans]'
old_format_dates = old_format_deep_copy.index.get_level_values(0)
old_format_deep_copy.index = pd.MultiIndex.from_arrays((old_format_dates,old_format_groups))
old_format_total = old_format_deep_copy[old_format_deep_copy.index.get_level_values(1) == 'Total [All Loans]']
old_format_fixed = old_format_deep_copy[old_format_deep_copy.index.get_level_values(1) == 'Fixed']
old_format_arm = old_format_deep_copy[old_format_deep_copy.index.get_level_values(1) == 'ARM']
old_format_final = pd.concat([old_format_total,old_format_fixed,old_format_arm])
old_format_final = old_format_final.sort_index(level=0)

# Stage 4: Perform the final concatenation of old- and new-format data, and save the result

In [12]:
delinquency_stats_final = pd.concat([old_format_final, new_format_final])
with open('../../data/mbs_data_pickled/delinquency_stats_balances.pkl','wb') as f:
    pickle.dump(delinquency_stats_final,f)

In [13]:
delinquency_stats_final

Current (Balance) Delinq 1 Month (Balance)  \
Date                                                                      
2007-01-25 ARM                        0.987345                 0.012655   
           Fixed                      0.989205                 0.010795   
           Total [All Loans]             0.989                    0.011   
2007-02-26 ARM                        0.963245                  0.02848   
           Fixed                      0.981958                 0.011405   
...                                        ...                      ...   
2020-02-25 Fixed                      0.793058                 0.021916   
           Total [All Loans]          0.756075                 0.027589   
2020-03-25 ARM                        0.733154                 0.032739   
           Fixed                      0.791816                 0.026877   
           Total [All Loans]          0.759422                 0.030114   

                             Delinq 2 Months (Balance)  \
Date                                                     
2007-01-25 ARM                                     0.0   
           Fixed                                   0.0   
           Total [All Loans]                       0.0   
2007-02-26 ARM                                0.008077   
           Fixed                              0.006663   
...                                                ...   
2020-02-25 Fixed                              0.017048   
           Total [All Loans]                  0.013918   
2020-03-25 ARM                                0.009103   
           Fixed                              0.013985   
           Total [All Loans]                  0.011289   

                             Delinq 3+ Months (Balance) Bankruptcy (Balance)  \
Date                                                                           
2007-01-25 ARM                                      0.0                  0.0   
           Fixed                                    0.0                  0.0   
           Total [All Loans]                        0.0                  0.0   
2007-02-26 ARM                                      0.0             0.000205   
           Fixed                                    0.0                  0.0   
...                                                 ...                  ...   
2020-02-25 Fixed                               0.016059             0.029396   
           Total [All Loans]                   0.020746             0.056436   
2020-03-25 ARM                                 0.025707             0.079814   
           Fixed                               0.015053             0.033695   
           Total [All Loans]                   0.020936             0.059163   

                             Foreclosure (Balance) REO (Balance)  \
Date                                                               
2007-01-25 ARM                                 0.0           0.0   
           Fixed                               0.0           0.0   
           Total [All Loans]                   0.0           0.0   
2007-02-26 ARM                                 0.0           0.0   
           Fixed                               0.0           0.0   
...                                            ...           ...   
2020-02-25 Fixed                          0.101298      0.021224   
           Total [All Loans]              0.105868      0.019368   
2020-03-25 ARM                            0.103059      0.016424   
           Fixed                           0.09268      0.025892   
           Total [All Loans]              0.098411      0.020664   

                             Total Mortgage Principal Remaining  
Date                                                             
2007-01-25 ARM                                     779339939.47  
           Fixed                                   253480360.74  
           Total [All Loans]                      1143360978.59  
2007-02-26 ARM                            

## TEST CODE

In [14]:
new_format_all_data

Current (Count) Current (Balance)  \
Date       Group                                                 
2013-06-25 Total - All Loans             NaN          0.509171   
           Total - Fixed                 NaN          0.636096   
           Total - ARM                   NaN          0.448502   
2013-07-25 Total - All Loans             NaN          0.512724   
           Total - Fixed                 NaN          0.647291   
...                                      ...               ...   
2020-02-25 Total - Fixed                 NaN          0.793058   
           Total - ARM                   NaN          0.726147   
2020-03-25 Total - All Loans             NaN          0.759422   
           Total - Fixed                 NaN          0.791816   
           Total - ARM                   NaN          0.733154   

                             Delinq 1 Month (Count) Delinq 1 Month (Balance)  \
Date       Group                                                               
2013-06-25 Total - All Loans                    NaN                 0.037624   
           Total - Fixed                        NaN                  0.03919   
           Total - ARM                          NaN                 0.036875   
2013-07-25 Total - All Loans                    NaN                 0.036741   
           Total - Fixed                        NaN                 0.035639   
...                                             ...                      ...   
2020-02-25 Total - Fixed                        NaN                 0.021916   
           Total - ARM                          NaN                  0.03218   
2020-03-25 Total - All Loans                    NaN                 0.030114   
           Total - Fixed                        NaN                 0.026877   
           Total - ARM                          NaN                 0.032739   

                             Delinq 2 Months (Count)  \
Date       Group                                       
2013-06-25 Total - All Loans                     NaN   
           Total - Fixed                         NaN   
           Total - ARM                           NaN   
2013-07-25 Total - All Loans                     NaN   
           Total - Fixed                         NaN   
...                                              ...   
2020-02-25 Total - Fixed                         NaN   
           Total - ARM                           NaN   
2020-03-25 Total - All Loans                     NaN   
           Total - Fixed                         NaN   
           Total - ARM                           NaN   

                             Delinq 2 Months (Balance)  \
Date       Group                                         
2013-06-25 Total - All Loans                   0.02013   
           Total - Fixed                      0.023596   
           Total - ARM                        0.018473   
2013-07-25 Total - All Loans                  0.023138   
           Total - Fixed                         0.026   
...                                                ...   
2020-02-25 Total - Fixed                      0.017048   
           Total - ARM                        0.011384   
2020-03-25 Total - All Loans                  0.011289   
           Total - Fixed                      0.013985   
           Total - ARM                        0.009103   

                             Delinq 3+ Months (Count)  \
Date       Group                                        
2013-06-25 Total - All Loans                      NaN   
           Total - Fixed                          NaN   
           Total - ARM                            NaN   
2013-07-25 Total - All Loans                      NaN   
           Total - Fixed                          NaN   
...                                               ...   
2020-02-25 Total - Fixed                          NaN   
           Total - ARM                            NaN   
2020-03-25 Total - All Loans                      NaN   
           Total -

In [15]:
next_test = old_format_counts_only.iloc[0:90].copy()
months = next_test.index.get_level_values(0)
months = months.drop_duplicates(keep='first')
for month in months:
    set_up_totals(next_test, month)
next_test.xs('2007-02-26',level=0)

NameError: name 'old_format_counts_only' is not defined

In [ ]:
jan_test = old_format_counts_only.xs('2007-01-25',level=0)
#jan_test.reset_index(inplace=True)

In [ ]:
# Idea from https://songhuiming.github.io/pages/2017/03/18/weighted-avarage-aggrefated-function-with-apply-and-agg/
# This yields the average percent of fixed-rate loans which are current
fixed_curr = sum(fixed_only['Current (Count)'] * fixed_only['Number of Loans']) / fixed_only['Number of Loans'].sum()
arm_curr = sum(arm_only['Current (Count)'] * arm_only['Number of Loans']) / arm_only['Number of Loans'].sum()
print(fixed_curr, arm_curr)

NameError: name 'fixed_only' is not defined

In [ ]:
new_format_all_data

Current (Count) Current (Balance)  \
Date       Group                                                 
2013-06-25 Total - All Loans        0.559372               NaN   
           Total - Fixed            0.674603               NaN   
           Total - ARM               0.49142               NaN   
2013-07-25 Total - All Loans        0.563772               NaN   
           Total - Fixed            0.686327               NaN   
...                                      ...               ...   
2020-02-25 Total - Fixed            0.842742               NaN   
           Total - ARM              0.736934               NaN   
2020-03-25 Total - All Loans        0.792293               NaN   
           Total - Fixed            0.839757               NaN   
           Total - ARM              0.751313               NaN   

                             Delinq 1 Month (Count) Delinq 1 Month (Balance)  \
Date       Group                                                               
2013-06-25 Total - All Loans              0.0392542                      NaN   
           Total - Fixed                  0.0383598                      NaN   
           Total - ARM                    0.0397816                      NaN   
2013-07-25 Total - All Loans              0.0401985                      NaN   
           Total - Fixed                  0.0335121                      NaN   
...                                             ...                      ...   
2020-02-25 Total - Fixed                  0.0201613                      NaN   
           Total - ARM                    0.0383275                      NaN   
2020-03-25 Total - All Loans              0.0291353                      NaN   
           Total - Fixed                  0.0223124                      NaN   
           Total - ARM                    0.0350263                      NaN   

                             Delinq 2 Months (Count)  \
Date       Group                                       
2013-06-25 Total - All Loans               0.0196271   
           Total - Fixed                   0.0198413   
           Total - ARM                     0.0195008   
2013-07-25 Total - All Loans                 0.02134   
           Total - Fixed                   0.0241287   
...                                              ...   
2020-02-25 Total - Fixed                    0.016129   
           Total - ARM                     0.0139373   
2020-03-25 Total - All Loans                0.012218   
           Total - Fixed                   0.0162272   
           Total - ARM                    0.00875657   

                             Delinq 2 Months (Balance)  \
Date       Group                                         
2013-06-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
           Total - ARM                             NaN   
2013-07-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
...                                                ...   
2020-02-25 Total - Fixed                           NaN   
           Total - ARM                             NaN   
2020-03-25 Total - All Loans                       NaN   
           Total - Fixed                           NaN   
           Total - ARM                             NaN   

                             Delinq 3+ Months (Count)  \
Date       Group                                        
2013-06-25 Total - All Loans                0.0829244   
           Total - Fixed                    0.0608466   
           Total - ARM                      0.0959438   
2013-07-25 Total - All Loans                 0.120596   
           Total - Fixed                    0.0844504   
...                                               ...   
2020-02-25 Total - Fixed                    0.0100806   
           Total - ARM                      0.0243902   
2020-03-25 Total - All Loans                0.0206767   
           Total -

In [ ]:
# Test the new-format one-month extraction function
extract_new_format_month('/Users/Alex/Library/Group Containers/UBF8T346G9.Office/BSABS-HE10_INVESTOR_REPORTS/' + \
    'June 2013 Through March 2020/', 'bear-stearns-2006-he10-investor-report-11-25-2016.xls')

,Date,Current (Count),Current (Balance),Delinq 1 Month (Count),Delinq 1 Month (Balance),Delinq 2 Months (Count),Delinq 2 Months (Balance),Delinq 3+ Months (Count),Delinq 3+ Months (Balance),Bankruptcy (Count),Bankruptcy (Balance),Foreclosure (Count),Foreclosure (Balance),REO (Count),REO (Balance),Number of Loans
Group,,,,,,,,,,,,,,,,
Total - All Loans,2016-11-25,0.673469,NaN,0.0333333,NaN,0.0163265,NaN,0.0741497,NaN,0.0408163,NaN,0.136735,NaN,0.0251701,NaN,1470
Total - Fixed,2016-11-25,0.764228,NaN,0.0325203,NaN,0.0211382,NaN,0.0617886,NaN,0.0292683,NaN,0.0796748,NaN,0.0113821,NaN,615
Total - ARM,2016-11-25,0.608187,NaN,0.0339181,NaN,0.0128655,NaN,0.0830409,NaN,0.0491228,NaN,0.177778,NaN,0.0350877,NaN,855


In [ ]:
# Old test of the old-format extraction function
extract_jan07_may13('2007 Through May 2013')

Current (Count)  \
Date       Group                                                         
2007-01-25 Total [All Loans]                                    0.9897   
           Group I Loans - Total                                0.9863   
           Group I Loans Fixed 1st Lien                         0.9891   
           Group I Loans Fixed 2nd Lien                         0.9519   
           Group I Loans 228 ARM                                0.9885   
...                                                                ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                   0.9095   
           Group II Loans Subgroup II 327 ARM                   0.9474   
           Group II Loans Subgroup III Fixed 1st Lien           0.9495   
           Group II Loans Subgroup III 228 ARM                  0.8780   
           Group II Loans Subgroup III 327 ARM                  0.8864   

                                                       Current (Balance)  \
Date       Group                                                           
2007-01-25 Total [All Loans]                                      0.9890   
           Group I Loans - Total                                  0.9859   
           Group I Loans Fixed 1st Lien                           0.9890   
           Group I Loans Fixed 2nd Lien                           0.9443   
           Group I Loans 228 ARM                                  0.9858   
...                                                                  ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                     0.8977   
           Group II Loans Subgroup II 327 ARM                     0.9646   
           Group II Loans Subgroup III Fixed 1st Lien             0.9604   
           Group II Loans Subgroup III 228 ARM                    0.8864   
           Group II Loans Subgroup III 327 ARM                    0.8986   

                                                       Delinq 1 Month (Count)  \
Date       Group                                                                
2007-01-25 Total [All Loans]                                           0.0103   
           Group I Loans - Total                                       0.0137   
           Group I Loans Fixed 1st Lien                                0.0109   
           Group I Loans Fixed 2nd Lien                                0.0481   
           Group I Loans 228 ARM                                       0.0115   
...                                                                       ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                          0.0401   
           Group II Loans Subgroup II 327 ARM                          0.0316   
           Group II Loans Subgroup III Fixed 1st Lien                  0.0152   
           Group II Loans Subgroup III 228 ARM                         0.0820   
           Group II Loans Subgroup III 327 ARM                         0.0227   

                                                       Delinq 1 Month (Balance)  \
Date       Group                                                                  
2007-01-25 Total [All Loans]                                             0.0110   
           Group I Loans - Total                                         0.0141   
           Group I Loans Fixed 1st Lien                                  0.0110   
           Group I Loans Fixed 2nd Lien                                  0.0557   
           Group I Loans 228 ARM                                         0.0142   
...                                                                         ...   
2007-06-25 Group II Loans Subgroup II 228 ARM                            0.0427   
           Group II Loans Subgroup II 327 ARM                            0.0133   
           Group II Loans Subgroup III Fixed 1st Lien                    0.0075   
           Group II Loans Subgroup III 228 ARM                           0.0722   
           Group II Loans Subgroup III 327 ARM       

In [ ]:
jan_test[jan_test.Group.str.contains('Fixed')]['Current (Count)'].multiply(jan_test[jan_test.Group.str.contains('Fixed')]['Number of Loans'] / fixed_num).sum()

0.9879837974683544

In [ ]:
# OLD IDEA
fixed_num = jan_test[jan_test.Group.str.contains('Fixed')]['Number of Loans'].sum()
fixed_only = jan_test[jan_test.Group.str.contains('Fixed')]
arm_num = jan_test[jan_test.Group.str.contains('ARM')]['Number of Loans'].sum()
arm_only = jan_test[jan_test.Group.str.contains('ARM')]

## Idea for New Format structure
- get_data function allocates items to the data_list and concatenates it at the end
- get_data calls extract_one_month and passes in the filename, and extract_one_month returns the data for that month
- extract_one_month chooses which row-location dictionaries to use based on the date in the filename